In [10]:
import MKLpy
import pandas as pd
import numpy as np
import sklearn.svm as svm
import pickle
import os
from sklearn.metrics.pairwise import rbf_kernel
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import KFold, cross_val_score
from sklearn.multiclass import OneVsRestClassifier  # support from multiclass
scaler = StandardScaler()
import time
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
from collections import defaultdict
import matplotlib.pyplot as plt
from MKLpy.preprocessing import normalization, rescale_01
from MKLpy.model_selection import cross_val_score, cross_val_predict
import pickle as pkl
###


from MKLpy.metrics import pairwise
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
#evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score 
from sklearn.metrics import roc_curve, auc


In [11]:
def open_pickle_file(path, pickle_file):
    file_loc = os.path.join(path, pickle_file)
    pickle_to_file = pickle.load(open(file_loc, "rb"))
    return pickle_to_file
def checkDir(dirLoc):
    print
    if os.path.exists(dirLoc):#Checks if the dir exists
        print("The directory exists")
    else:
        print ("No directory found for "+dirLoc) #Output if no directory
#         print
#         os.makedirs(CheckDir)#Creates a new dir for the given name
#         print "Directory created for "+CheckDir



In [113]:
hardDrivesLoc = '/media/ak/'
hardDrivesList = os.listdir('/media/ak/')

dataOnlyDrive = '/media/ak/DataOnly'  # external date only drive
#ext_drive_loc = ('/media/ak/My Passport/Experiment Data/')
data_dir = os.getenv('FINANCE_DATA') #internal folder with finance data 
folderList =  [s for s in os.listdir(dataOnlyDrive) if s.startswith('Dat') or s.startswith('Fin')]
# inputDrive = data_only_drive
# features = os.path.join(inputDrive,'features')
# labels = os.path.join(inputDrive, 'labels')

# symbols = [s for s in os.listdir(features) if s.endswith('.L')]
folderIdx = 0
finalLocation= "/".join((dataOnlyDrive,folderList[folderIdx]))
symbols = [s for s in os.listdir(finalLocation) if s.endswith('.L')] #keep a list of the symbols
os.listdir(finalLocation)
print(data_dir)

/home/ak/FinData/


In [13]:
locIdx= 1 #'''WorkDrive'''

selection = hardDrivesList[locIdx]

selectionLoc = os.path.join(hardDrivesLoc,selection ) 
''' location of WorkDrive'''
dataList = [s for s in os.listdir(selectionLoc) if s.startswith('Dat')]
DataLoc = os.path.join(hardDrivesLoc,selection,dataList[1] )
path= 'MKL_Experiments'
MKLExpPath = os.path.join(DataLoc,path)
#os.makedirs(os.path.join(DataLoc,path)) # run once so comment out afterwards- we can write an if statement later
checkDir(MKLExpPath)

The directory exists


## pick a symbol ## 

In [14]:
##picking up a specific feature
symbolIdx = 1 #pick one of the symbols
#symbols[symbolIdx] -->output :PRU.L
print(symbols[symbolIdx])
# do a join to get the location
symbolLocation = "/".join((finalLocation,symbols[symbolIdx])) 

# get he features now
symbolFeaturesLocation = "/".join((symbolLocation,'MODEL_BASED')) # where all the HMM output is

print(symbolFeaturesLocation) # <-- all the HMM model output is here, for each model there is a Date Folder and then OOS files
MKLSymbolPath = os.path.join(MKLExpPath,symbols[symbolIdx])

PRU.L
/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED


In [15]:
#os.makedirs(os.path.join(MKLExpPath,symbols[symbolIdx])) #make MKL path for symbol


### pick Features for a specific HMM MODEL Date ###
### then move to labels ...pick the location of the labels and make a list of all the available labels ##

In [86]:

symbolHMMDatesList = os.listdir(symbolFeaturesLocation) 

#list of all the MODEL dates we have generated features files for. each #
# each of these dates in symbolFeaturesDates corresponds to a list of dates
## (symbolHMMDatesList = '20170829', '20170710', '20170801', ... ]
#location of labels : /media/ak/DataOnly/FinDataReal/Labels/[Symbol :PRU.L]/NON_DIRECTIONAL

symbolLabelsLocation = "/".join((finalLocation, 'Labels',symbols[symbolIdx],'NON_DIRECTIONAL')) 

#list of all the label dates

symbolLabelsDates =[dateFile.split(".")[0] for dateFile in os.listdir(symbolLabelsLocation)]

# Output symbolLabelsDates --> ['20170704', '20180226', '20180208',...] all we are doing is going for this 
# os.listdir(symbolFeaturesLocation)


In [70]:
idxHMM=2
for idx,idxHMM in enumerate(sorted(symbolHMMDatesList)):
    print(idx)
    fileToCheck = os.path.join(symbolFeaturesLocation, symbolHMMDatesList[idx])
    checkDir(fileToCheck)
#     s

0
The directory exists
1
The directory exists
2
The directory exists
3
The directory exists
4
The directory exists
5
The directory exists
6
The directory exists
7
The directory exists
8
The directory exists
9
The directory exists
10
The directory exists
11
The directory exists
12
The directory exists
13
The directory exists
14
The directory exists
15
The directory exists
16
The directory exists
17
The directory exists
18
The directory exists
19
The directory exists
20
The directory exists
21
The directory exists
22
The directory exists
23
The directory exists
24
The directory exists
25
The directory exists
26
The directory exists
27
The directory exists
28
The directory exists
29
The directory exists
30
The directory exists
31
The directory exists
32
The directory exists
33
The directory exists
34
The directory exists
35
The directory exists
36
The directory exists
37
The directory exists
38
The directory exists
39
The directory exists
40
The directory exists
41
The directory exists
42

In [ ]:
##now lets go down into each HMM-model date, and pick all the forward futures (out of sample)
hmmFeatureLocations ={} #symbol-hmm-model-date index --> this is the indexation in symbolFeaturesDatesList
commonDatesDict ={} # this is a struct that will contain for each HMM date, the common labels/features- this should be used for training and testing
createDate =[] #place holder for the hash key of when the features got created
symbolEachModelFeaturesDates ={}
HMMModelFeaturesLabelsCommon ={} #location dictionary with 2 keys: HMM Date and Common Date
commonDates =[]

LocDictsList=[]

#this symbolFeaturesDatesList[featrsIdx] will give you a date: 20170710 =which contains all the HMM induced featureb
for hmmDateIdx, hmmDate in enumerate(sorted(symbolHMMDatesList)):
    
    symbolModelFeaturesDate = os.path.join(symbolFeaturesLocation, symbolHMMDatesList[hmmDateIdx])
    createDate = os.listdir(symbolModelFeaturesDate)[0].split("_")[7] #stupid hack
#     #output looks like this: /media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170710
    symbolEachModelFeaturesDates[symbolHMMDatesList[hmmDateIdx]]=[file.split("_")[5] for file in os.listdir(symbolModelFeaturesDate )] 
#      #output is a dictionary where the keys are the HMM models dates and the values a list of dates - for each HMM date we have a list of features
    
    for keyHMMDate in sorted(list(symbolEachModelFeaturesDates.keys())): # for each of the HMM model dates
        print(keyHMMDate)
        EachModelFeaturesDates = set(symbolEachModelFeaturesDates[keyHMMDate])
        commonDates=(list(EachModelFeaturesDates & set(symbolLabelsDates)))  
        #take the list of feature dates (conditional on HMM model date) + the list of labels -intersection!
        
        ''' we now produce a dict for each HMM model, where each value is a list of common dates and we are key-ed by the HMM Date'''
        commonDatesDict[keyHMMDate] = commonDates 
        for commonDate in commonDates:
            ''' iterate through all the common dates and figure out the location of each file for labels and features'''
            labelsCommonFileLoc = "/".join((symbolLabelsLocation, ".".join((commonDate,'csv'))))
#             comnDateFeatureLocMaster = os.path.join((symbolModelFeaturesDate, commonDate))
            commonDatesFeatureFile = "".join((symbols[symbolIdx],'_3_states_features_date:_',commonDate,"_now:_",createDate,"_.pickle"))
            FeatureFileLoc = os.path.join(symbolModelFeaturesDate,commonDatesFeatureFile)
            checkDir(FeatureFileLoc)
            checkDir(labelsCommonFileLoc)
            conditions=[os.path.exists(FeatureFileLoc), os.path.exists(labelsCommonFileLoc)]
            print(conditions)            
            if all(conditions)== True:
                print('all good on Date:', commonDate)
                HMMModelFeaturesLabelsCommon[keyHMMDate, commonDate] =[FeatureFileLoc,labelsCommonFileLoc ]
                hmmFeatureLocations[keyHMMDate, commonDate] =FeatureFileLoc
                
                
            else:
                print('problem on date: ', commonDate)
                continue 

LocDictsList = [commonDatesDict, HMMModelFeaturesLabelsCommon, hmmFeatureLocations]
LocDictsListFile = "/".join((MKLSymbolPath,"LocDictsList.pkl"))
pkl.dump( LocDictsList, open( LocDictsListFile, "wb" ) )    # dump it all in the same location    

In [112]:

# #     commonDates =list(set(symbolEachModelFeaturesDates) & set(symbolLabelsDates)) # this is a list of features and labels dates
# #     HMMModelFeaturesLabelsCommon[hmmDate] = commonDates
# for HMMDate, commonDate in zip(symbolHMMDatesList,commonDates):
#     print(HMMDate, commonDate)
# #     print(HMMModelFeaturesLabelsCommon[HMMDate,commonDate])

len(EachModelFeaturesDates)
len(sorted(symbolHMMDatesList))

89

In [ ]:
# for HMMdate in np.sort(symbolHMMDatesList):
#     print(len(list(commonDatesDict[HMMdate])))

In [32]:
def forwardDates(idxKey, commonLocs):
    ''' return all the forward looking dates for each idxKey we use for training'''
    keys =list(commonLocs.keys()) #this produces a list with 2 keys -first HMM Symbol Date and common Date
    lookAheadKeys = sorted(i for i in keys if i>keys[idxKey])
    return dict((k, commonLocs[k]) for k in lookAheadKeys )

def featureCreation(locDict, idxKey):
    ''' gives out clean features and labels for a given locDict and a idxKey '''
    keys=list(locDict.keys())
    featuresIdxDirFileLoc= locDict[keys[idxKey]][0]
    labelsIdxDirFileLoc= locDict[keys[idxKey]][1]
    ''' read the features file'''
    featuresTupleFile = pickle.load(open(featuresIdxDirFileLoc,"rb"), encoding='latin1') 
    dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1],\
                                                 featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)
    ''' read the labels file'''
    labelsDf=pd.read_csv(labelsIdxDirFileLoc)
    ''' pop the labels out'''
    labels =labelsDf['label_PrMov__window_5__thres_arbitrary__0.1'] 
    '''dataframe of Features and Labels - X and Y'''
    dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
    labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])
    ''' drop the labels from the features'''
    dfX = dfXY.drop(columns=[ labelName])
    arrX = np.array(dfX)
    ''' feature normalisation'''
    #feature scaling in [0,1] - X = rescale_01(arrX)
    X = normalization(rescale_01(arrX))
    y = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]
    ''' returns features, labels'''
    return X, y

In [42]:
### testing ###
keys =sorted(list(HMMModelFeaturesLabelsCommon.keys())) #make a sorted list of all the locations
idxKey=22 #pick a key
print(keys[idxKey]) #print that key
sorted(i for i in keys if i>keys[idxKey])
forwardDatesKeysList =list(forwardDates(idxKey = 22, commonLocs=HMMModelFeaturesLabelsCommon).keys())

('20170116', '20170717')


In [47]:
checkDir(HMMModelFeaturesLabelsCommon[keys[idxKey]][0])

No directory found for /media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20180419/PRU.L_3_states_features_date:_20170717_now:_20181229_.pickle


In [ ]:
#do this to store commonLocs in the appropriate folder
commonLocsSymbolPath = os.path.join(MKLSymbolPath, 'commonLocs.pickle')
with open(commonLocsSymbolPath, 'wb') as handle:
    pickle.dump(commonLocs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(commonLocsSymbolPath, 'rb') as handle:
    b = pickle.load(handle)
commonLocs == b # simple check- delete

###Functions for Training Set ###

In [ ]:
# all the keys the model fitting key - can use something like this to test?!


In [ ]:
## testing above##
forwardDatesDict = forwardDates(idxKey = 3, commonLocs=commonLocs)
for forwardDateKey in list(forwardDatesDict.keys()):
    checkDir(forwardDatesDict[forwardDateKey][0])


In [ ]:
os.listdir('/media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/')

## do Training of 3 models ##

In [ ]:
trainingIdxKey =2

Xtr, ytr =  featureCreation(locDict = commonLocs, idxKey =trainingIdxKey)

print('checking shapes',Xtr.shape[0]==ytr.shape[0])
if (Xtr.shape[0]==ytr.shape[0]):
    print('Shapes Match- starting training ')
    ##polynomial Kernels ##
    KLtr = [pairwise.homogeneous_polynomial_kernel(Xtr, degree=d) for d in range(4)]
    ''' Compute RBF Kernels'''
    gamma_range = np.logspace(-9, 3, 13)
    ker_list = [rbf_kernel(Xtr, gamma=g) for g in gamma_range]

    ### and train 3 classifiers ###
    clf = AverageMKL().fit(KLtr,ytr) #a wrapper for averaging kernels
    print(clf.weights) #print the weights of the combination of base kernels
    print ('training EasyMKL...for polynomials and RBF', end='')
    clfEasy = EasyMKL(lam=0.1).fit(KLtr,ytr)#combining kernels with the EasyMKL algorithm
    clfRBF = EasyMKL(lam=0.1).fit(ker_list,ytr)  
    print('finished training')     
else:
    print('Shapes dont match.')
    pass

In [ ]:
range(trainingIdxKey+1,trainingIdxKey+5)

### simple testing ###

In [ ]:
testKernelsPolyn ={}
testKernelsRBF ={}


forwardDatesDict = forwardDates(trainingIdxKey, commonLocs)
testingIdxKey =13 #need to print the date to be precise here
for testingIdxKey in range(trainingIdxKey+1,trainingIdxKey+5):
    Xte, Yte=featureCreation(forwardDatesDict , testingIdxKey)
    print('checking shapes',Xte.shape[0]==Yte.shape[0])
    if (Xte.shape[0]==Yte.shape[0]):
        ##polynomial Kernels ##
        print('Computing test case')
        KLte = [pairwise.homogeneous_polynomial_kernel(Xte,Xtr, degree=d) for d in range(4)]
        ''' Compute RBF Kernels'''
        gamma_range = np.logspace(-9, 3, 13)
        ker_list_te = [rbf_kernel(Xte, gamma=g) for g in gamma_range]
        testKernelsPolyn[testingIdxKey] =[KLte, Yte]
        testKernelsRBF[testingIdxKey] =[ker_list_te, Yte]
    else:
        print('Shapes Dont match- move to next date')
        continue

In [ ]:
print('Average Kernel Testing')
y_pred = clf.predict(KLte)                 #predictions
y_score = clf.decision_function(KLte)      #rank
accuracy = accuracy_score(Yte, y_pred)

print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))
fprAverage, tprAverage, thresholdsAverage = roc_curve(Yte.ravel(), y_score.ravel())
roc_auc = auc(fprAverage, tprAverage)

''' Test Linear'''
print('MKL-Linear Testing')
y_predMKLLinear = clfEasy.predict(KLte)                 #predictions
y_scoreMKLLinear = clfEasy.decision_function(KLte)  #rank

accuracy_MKLLinear = accuracy_score(Yte, y_predMKLLinear)
roc_auc_MKLLinear = roc_auc_score(Yte, y_scoreMKLLinear)
print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy_MKLLinear, roc_auc_MKLLinear))

fprLinear, tprLinear, thresholds = roc_curve(Yte.ravel(), y_scoreMKLLinear.ravel())

roc_auc_Linear = auc(fprLinear, tprLinear)

# fprMKLLinear, fprMKLLinear, thresholdsRBF =roc_curve(Yte, y_scoreMKLLinear)

print(roc_auc_Linear)

_model_results= {
    'clfs': np.empty((M, T)),
    'test_F1': np.empty((M, T)),
    'data_date': np.empty((M, T)),
    'test_recall': np.empty((M, T)),
    'train_recall': np.empty((M, T)),
    'test_accuracy' :np.empty((M, T)),
    'train_accuracy' :np.empty((M, T)),
}
    _model_results['test_accuracy'][_idx, :] = accuracy_score(y_test, y_predict)
    _model_results['test_recall'][_idx, :] = recall_score(y_true=y_test, y_pred=y_predict)
    _model_results['train_accuracy'][_idx, :] = accuracy_score(y_train, y_predict_train)
    _model_results['train_recall'][_idx, :] = recall_score(y_true=y_train, y_pred=y_predict_train)
    _model_results['test_F1'][_idx, :]=f1_score(y_true=y_test, y_pred=y_predict)

In [ ]:
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.plot(fprAverage, tprAverage, color='darkred',label='Average Kernel')
plt.legend(loc="lower right")
plt.plot(fprLinear, tprLinear, color='orange',label='MKL Polyn')
plt.legend(loc="lower right")
lw=2
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--',label='ROC curve (area = %0.2f)' % roc_auc)


plt.plot([0, 1], [0, 1], color='orange', lw=lw, linestyle='-',label='ROC curve (area = %0.2f)' % roc_auc_Linear)
plt.legend(loc="lower left")

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example (MKL)')

plt.show()

In [ ]:

#for idxKey, commonDate in #enumerate(commonLocs.keys()):
for idxKey in range(3):
   
        print('Doing key:')
        print(keys[idxKey])
        ''' locations'''
        featuresIdxDirFileLoc= commonLocs[keys[idxKey]][0]
        labelsIdxDirFileLoc= commonLocs[keys[idxKey]][1]  

        featuresTupleFile = pickle.load(open(featuresIdxDirFileLoc,"rb"), encoding='latin1')   
        ''' read the labels file'''
        labelsDf=pd.read_csv(labelsIdxDirFileLoc)
        ''' take in the input file'''
        dfFeatures = pd.concat([featuresTupleFile[0], featuresTupleFile[1],\
                                                     featuresTupleFile[2], featuresTupleFile[3]], axis=1, sort=False).fillna(0)
        #take the labels out
        labels =labelsDf['label_PrMov__window_5__thres_arbitrary__0.1'] 
        print ('preprocessing data...', end='')
        '''dataframe of Features and Labels - X and Y'''
        dfXY = pd.concat([dfFeatures, labels], axis=1, sort='False').dropna()
        labelName = str(dfXY.columns[dfXY.columns.str.contains(pat='label')].values[0])
        ''' drop the labels from the features'''
        dfX = dfXY.drop(columns=[ labelName])
        print("Shape of dfX..",dfX.shape[0])   
        ''' MKL stuff- convert the Dataframe into an array'''
        arrX = np.array(dfX)
        ''' feature normalisation'''
         #feature scaling in [0,1] - X = rescale_01(arrX)
        X = normalization(rescale_01(arrX))
        print (X.shape,'done')

        y = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]
        print("Shape of y..", y.shape)
        '''training and testing- this we can probably skip'''
        Xtr,Xte,Ytr,Yte = train_test_split(X,y, test_size=.1, random_state=42)

        '''compute homogeneous polynomial kernels with degrees 0,1,2,...,10'''

        print ('computing Homogeneous Polynomial Kernels...', end='')

        KLtr = [pairwise.homogeneous_polynomial_kernel(Xtr, degree=d) for d in range(4)]
        KLte = [pairwise.homogeneous_polynomial_kernel(Xte,Xtr, degree=d) for d in range(4)]

        ''' Compute RBF Kernels'''
        ### test set work ###
        #KLtest = [pairwise.homogeneous_polynomial_kernel(Xte,Xtr, degree=d) for d in range(4)]
        gamma_range = np.logspace(-9, 3, 13)
        ker_list = [rbf_kernel(Xtr, gamma=g) for g in gamma_range]

        print ('RBF Kernels done')

        print ('training AverageMKL...', end='')
        clf = AverageMKL().fit(KLtr,Ytr) #a wrapper for averaging kernels
   
        ''' need to serialise this--here'''
        print ('done')
        print(clf.weights) #print the weights of the combination of base kernels

        print ('training EasyMKL...for polynomials and RBF', end='')
        clfEasy = EasyMKL(lam=0.1).fit(KLtr,Ytr)#combining kernels with the EasyMKL algorithm
 
        clfRBF = EasyMKL(lam=0.1).fit(ker_list,Ytr)  
         
        #lam is a hyper-parameter in [0,1]
        print ('weights:')
        print (clfEasy.weights)
        print(clfRBF.weights)
        print('Training Done')
           
        
        ''' Now do the testing'''
        try: 
            print('Average Kernel Testing')
            y_pred = clf.predict(KLte)                 #predictions
            y_score = clf.decision_function(KLte)      #rank
            accuracy = accuracy_score(Yte, y_pred)
            roc_auc = roc_auc_score(Yte, y_score)
            print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))

            ''' Test Linear'''
            print('MKL-Linear Testing')
            y_predTest = clfEasy.predict(KLte)                 #predictions
            y_scoreTest = clfEasy.decision_function(KLte)      #rank
            accuracy = accuracy_score(Yte, y_predTest)
            roc_auc = roc_auc_score(Yte, y_scoreTest)
            print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))

            '''Test RBF'''
            print('MKL-RBF Testing')
            y_predRBF = clfRBF.predict(KLte)                 #predictions
            y_scoreRBF = clfRBF.decision_function(KLte)      #rank
            accuracyRBF = accuracy_score(Yte, y_predRBF)
            roc_aucRBF = roc_auc_score(Yte, y_scoreRBF)
            print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracyRBF, roc_aucRBF))
        except IndexError: #  catch the error      
            continue# pass will basically ignore it

    
    

In [ ]:
testDirectory = os.getcwd()
# save the model to disk
filename = os.path.join(testDirectory,'test_MKL_model.pkl')
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model

### pick up the train features ###

## split the data + standarise ##

In [ ]:
X =StandardScaler().fit_transform(dfX)
y = dfXY[dfXY.columns[dfXY.columns.str.contains(pat='label')]].iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print(X.shape, y.shape)

In [ ]:
# testX = StandardScaler().fit_transform(dfTestXY)
# testY = dfTestXY[dfTestXY.columns[dfTestXY.columns.str.contains(pat='label')]].iloc[:, 0]

## Simple Classifiers for Comparison ##

In [ ]:
y

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
y_binarised = label_binarize(y, classes=[0, 1])
n_classes = y_binarised.shape[1]
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Yte[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(Yte.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

#    y_score = clf.decision_function(KLte)      #rank
#     accuracy = accuracy_score(Yte, y_pred)
#     roc_auc = roc_auc_score(Yte, y_score)

In [ ]:
fpr, tpr, thresholds =roc_curve(Yte, y_score)

roc_auc = auc(fpr, tpr)

fprRBF, tprRBF, thresholdsRBF =roc_curve(Yte, y_scoreRBF)

roc_aucRBF = auc(fprRBF, tprRBF)
print(roc_aucRBF)

In [ ]:
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.plot(fpr, tpr, color='darkred',)
plt.plot(fprRBF, tprRBF, color='darkorange',)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--',label='ROC curve (area = %0.2f)' % roc_auc)
plt.legend(loc="lower right")
plt.plot([0, 1], [0, 1], color='blue', lw=lw, linestyle='--',label='ROC curve (area = %0.2f)' % roc_aucRBF)
plt.legend(loc="lower left")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example (MKL)')

plt.show()


In [ ]:
y_predRBF = clfRBF.predict(KLte)                 #predictions
y_scoreRBF = clfRBF.decision_function(KLte)      #rank
accuracyRBF = accuracy_score(Yte, y_predRBF)
roc_aucRBF = roc_auc_score(Yte, y_scoreRBF)

### Trying MKL for the very first time ###

In [ ]:
#preprocess data
print ('preprocessing data...', end='')
from MKLpy.preprocessing import normalization, rescale_01
arrX = np.array(dfX)
X = rescale_01(arrX) #feature scaling in [0,1]
X = normalization(rescale_01(arrX))
Y=y
print(X.shape, y.shape)
# ## test set
arrTestX = np.array(dfTestX)
TestX = rescale_01(arrTestX) #feature scaling in [0,1]
TestX = normalization(rescale_01(arrTestX))
TestY=testY

### splitting the date into training and testing ###

In [ ]:
Xtr,Xte,Ytr,Yte = train_test_split(X,Y, test_size=.55, random_state=42)
print (X.shape,'done')

In [ ]:
#compute homogeneous polynomial kernels with degrees 0,1,2,...,10.
print ('computing Homogeneous Polynomial Kernels...', end='')

from MKLpy.metrics import pairwise
KLtr = [pairwise.homogeneous_polynomial_kernel(Xtr, degree=d) for d in range(4)]
KLte = [pairwise.homogeneous_polynomial_kernel(Xte,Xtr, degree=d) for d in range(4)]
from sklearn.metrics.pairwise import rbf_kernel
gamma_range = np.logspace(-9, 3, 13)
ker_list = [rbf_kernel(Xtr, gamma=g) for g in gamma_range]
print ('RBF Kernels done')
### test set work ###
KLtest = [pairwise.homogeneous_polynomial_kernel(TestX,Xtr, degree=d) for d in range(4)]
from sklearn.metrics.pairwise import rbf_kernel
gamma_range = np.logspace(-9, 3, 13)
ker_list = [rbf_kernel(Xtr, gamma=g) for g in gamma_range]


In [ ]:
#clf.fit(X_train, y_train)
classifier = SVC(kernel='linear', C=0.01)
linear_clf=classifier.fit(Xtr, Ytr)
# Create a SVC classifier using an RBF kernel
rbf_classifier= SVC(kernel='rbf', random_state=0, gamma=1000, C=1000)
rbf_clf=rbf_classifier.fit(Xtr, Ytr)

In [ ]:
#MKL algorithms
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
print ('training AverageMKL...', end='')
clf = AverageMKL().fit(KLtr,Ytr) #a wrapper for averaging kernels
print ('done')
print(clf.weights) #print the weights of the combination of base kernels
K_average = clf.ker_matrix #the combined kernel matrix

In [ ]:
# clfRBF = clf = AverageMKL().fit(ker_list,Ytr) #a wrapper for averaging kernels
K_average_rbf = clfRBF.ker_matrix

In [ ]:
dfKaverage =pd.DataFrame(K_average)
dfKaverageRBF =pd.DataFrame(K_average_rbf)


In [ ]:

_=plt.matshow(dfKaverage, fignum=100)
plt.gca().set_aspect('auto')
plt.title(' K-average matrix' ,y=1.18)
plt.show()
plt.matshow(dfKaverageRBF, fignum=100)
plt.gca().set_aspect('auto')
_=plt.title('K-average RBF matrix', y=1.18)
plt.show()

In [ ]:
print ('training EasyMKL...', end='')
clf = EasyMKL(lam=0.1).fit(KLtr,Ytr)#combining kernels with the EasyMKL algorithm
clfEasyRBF = EasyMKL(lam=0.1).fit(ker_list,Ytr)
#lam is a hyper-parameter in [0,1]
print ('done')
print (clf.weights)

In [ ]:
_=plt.hist(clfEasyRBF.weights)
plt.show()
_=plt.hist (clf.weights)

In [ ]:
dfWeights =pd.DataFrame()

In [ ]:
dfweights = pd.DataFrame(clfEasyRBF.weights) # pd.DataFrame(dict(rate=np.random.randn(10000)))
dfweights2 = pd.DataFrame(clf.weights) #pd.DataFrame(dict(rate=np.random.randn(10000)))
dfweights.rename(columns={'0':'weights'})

fig, axes = plt.subplots(1, 2)

_=dfweights.hist( ax=axes[0],label='RBF')

_=dfweights2.hist( ax=axes[1])
plt.show()

In [ ]:
#evaluate the solution
from sklearn.metrics import accuracy_score, roc_auc_score 
y_pred = clf.predict(KLte)                 #predictions
y_score = clf.decision_function(KLte)      #rank
accuracy = accuracy_score(Yte, y_pred)
roc_auc = roc_auc_score(Yte, y_score)
print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))

In [ ]:
y_pred = clf.predict(KLte)                 #predictions
y_pred_linear = linear_clf.predict(Xte)
y_score = clf.decision_function(KLte)      #rank
accuracy = accuracy_score(Yte, y_pred)
roc_auc = roc_auc_score(Yte, y_score)
print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))

In [ ]:
y_predTest = clf.predict(KLtest)                 #predictions
y_scoreTest = clf.decision_function(KLtest)      #rank
accuracy = accuracy_score(TestY, y_predTest)
roc_auc = roc_auc_score(TestY, y_scoreTest)
print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))

In [ ]:
plt.plot(clf.decision_function(KLtr))

In [ ]:
# print ('tuning lambda for EasyMKL...', end='')
# base_learner = SVC(C=100)	#simil hard-margin svm
# best_results = {}
# for lam in [0, 0.01, 0.1, 0.2, 0.9, 1]:#possible lambda values for the EasyMKL algorithm
#     #MKLpy.model_selection.cross_val_predict performs the cross validation automatically, it optimizes the accuracy
#     #the counterpart cross_val_score optimized the roc_auc_score (use score='roc_auc')
#     #WARNING: these functions will change in the next version
#     scores = cross_val_predict(KLtr, Ytr, EasyMKL(learner=base_learner, lam=lam), n_folds=5, score='accuracy')
#     acc = np.mean(scores)
# #     if not best_results or best_results['score'] < acc:
# #         best_results = {'lam' : lam, 'score' : acc}